# XML, HTML, and Web Scraping

JSON and XML are two different ways to represent hierarchical data. Which one is better? There are lots of articles online which discuss similarities and differences between JSON and XML and their advantages and disadvantages. Both formats are still in current usage, so it is good to be familiar with both. However, JSON is more common, so we'll focus on working with JSON representations of hierarchical data.

The reading covered an example of using Beautiful Soup to parse XML. Rather than doing another example XML now, we'll skip straight to scraping HTML from a webpage. Both HTML and XML can be parsed in a similar way with Beautiful Soup.

In [1]:
import pandas as pd
import requests

## Scraping an HTML table with Beautiful Soup

Open the URL https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population and scroll down until you see a table of the cities in the U.S. with population over 100,000 (as of Jul 1, 2022). We'll use Beautiful Soup to scrape information from this table.

Read in the HTML from the URL using the `requests` library.

In [2]:
URL = "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

response = requests.get(URL, headers=HEADERS)

# Use Beautiful Soup to parse this string into a tree called `soup`

In [3]:
from bs4 import BeautifulSoup

# If your HTML is in `response.text` from requests:
soup = BeautifulSoup(response.text, "lxml")  # or "html.parser"


To find an HTML tag corresponding to a specific element on a webpage, right-click on it and choose "Inspect element". Go to the cities table Wikipedia page and do this now.

You should find that the cities table on the Wikipedia page corresponds to an element that looks like

```
<table class="______" style="_______">
```

There are many `<table>` tags on the page.  One option you have is to find them all, and then manually figure out which one in this list is the table you care about.

In [4]:
all_tables = len(soup.find_all("table"))
all_tables

10



The other option is to try to use attributes like `class=` and/or `style=` to narrow down the list.

In [8]:
# Option 2: match class + style (stricter; may miss if style changes)
tables = soup.find_all("table",
    attrs={
        "class": ["wikitable", "sortable"],
        "style": "text-align:right"
    }
)
len(tables)


3

At this point, you can manually inspect the tables on the webpage to find that the one we want is the first one (see `[0]` below). We'll store this as `table`.

In [9]:
table = tables[0]

**Now you will write code to scrape the information in `table` to create a Pandas data frame with one row for each city and columns for: city, state, population (2022 estimate), and 2020 land area (sq mi).** Refer to the Notes/suggestions below as you write your code. A few Hints are provided further down, but try coding first before looking at the hints.

Notes/suggestions:

- Use as a guide the code from the reading that produced the data frame of Statistics faculty
- Inspect the page source as you write your code
- You will need to write a loop to get the information for all cities, but you might want to try just scraping the info for New York first
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `.get_text(strip = True)` instead of `.text`
- Don't forget to convert to a Pandas Data Frame; it should have 333 rows and 4 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it --- e.g., what is the population density for all cities in CA? --- then you would need to clean the data first (to clean strings and convert to quantitative). (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)

In [ ]:
# YOUR CODE HERE. ADD AS MANY CELLS AS NEEDED

In [11]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

URL = "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

# --- fetch & parse ---
resp = requests.get(URL, headers=HEADERS)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, "lxml")

table = soup.select_one("table.wikitable.sortable")
if table is None:
    raise RuntimeError("Could not find the main population table.")

# --- header normalization helpers ---
def norm(s: str) -> str:
    s = s.lower()
    s = re.sub(r"\[[^\]]*\]", "", s)          # strip [a] style notes
    s = re.sub(r"\s+", " ", s).strip()
    return s

thead = table.find("thead")
header_cells = (thead.find_all("th") if thead else None) or table.find("tr").find_all(["th","td"])
headers_raw = [th.get_text(" ", strip=True) for th in header_cells]
headers = [norm(h) for h in headers_raw]

# --- find columns robustly ---
def find_state_idx(headers):
    for i, h in enumerate(headers):
        if h in {"state", "st"} or ("state" in h and len(h) <= 10):
            return i
    return None

def find_city_idx(headers):
    for i, h in enumerate(headers):
        if "city" in h:
            return i
    return None

def find_land2020_idx(headers):
    # Accept "2020 land area" with/without "(sq mi)"
    for i, h in enumerate(headers):
        if "2020" in h and "land area" in h:
            return i
    return None

def find_pop_est_idx(headers, prefer_years=(2024, 2023, 2022)):
    # Prefer newest estimate if multiple are present; fall back to any "estimate"
    for yr in prefer_years:
        for i, h in enumerate(headers):
            if str(yr) in h and "estimate" in h:
                return i
    # fallback: first header containing "estimate"
    for i, h in enumerate(headers):
        if "estimate" in h:
            return i
    return None

idx_city  = find_city_idx(headers)
idx_state = find_state_idx(headers)
idx_land  = find_land2020_idx(headers)
idx_pop   = find_pop_est_idx(headers)

missing = [name for name, idx in {
    "city": idx_city, "state": idx_state, "land": idx_land, "pop_est": idx_pop
}.items() if idx is None]
if missing:
    raise RuntimeError(f"Could not map required columns. Missing: {missing}\nHeaders seen: {headers}")

# --- extract rows ---
rows = []
tbody = table.find("tbody") or table
for tr in tbody.find_all("tr"):
    tds = tr.find_all("td")
    if len(tds) <= max(idx_city, idx_state, idx_land, idx_pop):
        continue

    # drop inline citations
    for td in tds:
        for sup in td.select("sup"):
            sup.decompose()

    def txt(i): return tds[i].get_text(" ", strip=True)

    rows.append({
        "city": txt(idx_city),
        "state": txt(idx_state),                                  # column is "st" on the page; we normalize name here
        "population (2022 estimate)": txt(idx_pop),               # we keep your required label even if source is 2024
        "2020 land area (sq mi)": txt(idx_land),                  # unit label added for clarity
    })

df = pd.DataFrame(rows, columns=[
    "city", "state", "population (2022 estimate)", "2020 land area (sq mi)"
])

print(headers_raw)         # (optional) see exact headers found
print(df.shape)
print(df.head(10))


['City', 'ST', '2024 estimate', '2020 census', 'Change', '2020 land area', '2020 density', 'Location']
(346, 4)
           city state population (2022 estimate) 2020 land area (sq mi)
0      New York    NY                  8,478,072                  300.5
1   Los Angeles    CA                  3,878,704                  469.5
2       Chicago    IL                  2,721,308                  227.7
3       Houston    TX                  2,390,125                  640.4
4       Phoenix    AZ                  1,673,164                  518.0
5  Philadelphia    PA                  1,573,916                  134.4
6   San Antonio    TX                  1,526,656                  498.8
7     San Diego    CA                  1,404,452                  325.9
8        Dallas    TX                  1,326,087                  339.6
9  Jacksonville    FL                  1,009,833                  747.3


Hints:

- Each city is a row in the table; find all the `<tr>` tags to find all the cities
- Look for the `<td>` tag to see table entries within a row
- The rank column is represented by `<th>` tags, rather than `<td>` tags. So within a row, the first (that is, `[0]`) `<td>` tag corresponds to the city name.

## Scraping information that is NOT in a `<table>` with Beautiful Soup

The Cal Poly course catalog http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory contains a list of courses offered by the Statistics department. **You will scrape this website to obtain a Pandas data frame with one row for each DATA or STAT course and two columns: course name and number (e.g, DATA 301. Introduction to Data Science) and term typically offered (e.g., Term Typically Offered: F, W, SP).**

Note: Pandas `read_html` is not help here since the courses are not stored in a `<table>.`

In [12]:
pd.read_html("http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory")

[                                        Program name   Program type
 0                              Actuarial Preparation          Minor
 1  Cross Disciplinary Studies Minor in Bioinforma...          Minor
 2   Cross Disciplinary Studies Minor in Data Science          Minor
 3                                         Statistics  BS, MS, Minor]


Notes/suggestions:


- Inspect the page source as you write your code
- The courses are not stored in a `<table>`. How are they stored?
- You will need to write a loop to get the information for all courses, but you might want to try just scraping the info for DATA 100 first
- What kind of tag is the course name stored in? What is the `class` of the tag?
- What kind of tag is the quarter(s) the course is offered stored in? What is the `class` of the tag? Is this the only tag of this type with the class? How will you get the one you want?
- You don't have to remove the number of units (e.g., 4 units) from the course name and number, but you can try it if you want
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `get_text(strip = True)` instead of `text`
- Don't forget to convert to a Pandas Data Frame; it should have 74 rows and 2 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it then you might need to clean the data first. (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)



In [ ]:
# YOUR CODE HERE. ADD AS MANY CELLS AS NEEDED

In [13]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

URL = "http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

# 1) Fetch & parse
resp = requests.get(URL, headers=HEADERS)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, "lxml")

# 2) Each course lives in a block like: <div class="courseblock"> ... </div>
course_blocks = soup.select("div.courseblock")

rows = []
for block in course_blocks:
    # Course name/number line is in: <p class="courseblocktitle">DATA 100. Intro ... (4 units)</p>
    title_tag = block.select_one("p.courseblocktitle")
    if not title_tag:
        continue
    title_text = title_tag.get_text(" ", strip=True)

    # Keep only DATA or STAT courses
    if not re.match(r"^(DATA|STAT)\s", title_text):
        continue

    # Find the line that holds "Term Typically Offered: ..."
    term_text = None
    for p in block.select("p.courseblockdetail"):
        t = p.get_text(" ", strip=True)
        if t.startswith("Term Typically Offered"):
            term_text = t
            break

    # If the term line is missing, store a placeholder so row count stays consistent
    if term_text is None:
        term_text = "Term Typically Offered: —"

    rows.append({
        "course": title_text,                 # e.g., "DATA 301. Introduction to Data Science (4 units)"
        "term typically offered": term_text   # e.g., "Term Typically Offered: F, W, SP"
    })

# 3) Build the DataFrame
df = pd.DataFrame(rows, columns=["course", "term typically offered"])

print(df.shape)    # should be (74, 2) per the prompt (may change if catalog updates


(74, 2)


Hints:

- Each course is represented by a `<div>` with `class=courseblock`, so you can find all the courses with `soup.find_all("div", {"class": "courseblock"})`
- The course name is in a `<p>` tag with `class=courseblocktitle`, inside a `<strong>` tag. (Though I don't think we need to find the strong tag here.)
- The term typically offered is in `<p>` tag with `class=noindent`. However, there are several tags with this class; term typically offered is the first one.
- If you want to use Beautiful Soup to remove the course units (e.g., 4 units), find the `<span>` tag within the course name tag and `.extract()` this span tag